<a href="https://colab.research.google.com/github/mannb986/life_expectancy_nn/blob/main/life_expectancy_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Neural Networks

The World Health Organization (WHO)’s Global Health Observatory (GHO) data repository tracks life expectancy for countries worldwide by following health status and many other related factors.

Although there have been a lot of studies undertaken in the past on factors affecting life expectancy considering demographic variables, income composition, and mortality rates, it was found that the effects of immunization and human development index were not taken into account.

This dataset covers a variety of indicators for all countries from 2000 to 2015 including:

* Immunization factors
* Mortality factors
* Economic factors
* Social factors
* other health-related factors

Ideally, this data will eventually inform countries concerning which factors to change in order to improve the life expectancy of their populations. If we can predict life expectancy well given all the factors, this is a good sign that there are some important patterns in the data. Life expectancy is expressed in years, and hence it is a number. This means that in order to build a predictive model one needs to use regression.

In this project I will design, train and evaluate a neural network model performing the task of regression to predict the life expectancy of countries using this dataset.

## Data Loading & Observing

In [1]:
from google.colab import files


uploaded = files.upload()

Saving life_expectancy.csv to life_expectancy.csv


In [2]:
import pandas as pd

dataset = pd.read_csv('life_expectancy.csv')

In [3]:
dataset.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,Afghanistan,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,Afghanistan,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,Afghanistan,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,Afghanistan,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,Afghanistan,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


In [4]:
dataset.describe()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2.938000e+03,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,2007.518720,164.725664,30.303948,4.546875,738.251295,83.022124,2419.592240,38.381178,42.035739,82.617767,5.924098,82.393125,1.742103,6611.523863,1.023085e+07,4.821886,4.852144,0.630362,12.009837,69.234717
std,4.613841,124.086215,117.926501,3.921946,1987.914858,22.996984,11467.272489,19.935375,160.445548,23.367166,2.400770,23.655562,5.077785,13296.603449,5.402242e+07,4.397621,4.485854,0.205140,3.265139,9.509115
min,2000.000000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.370000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000,36.300000
25%,2004.000000,74.000000,0.000000,1.092500,4.685343,82.000000,0.000000,19.400000,0.000000,78.000000,4.370000,78.000000,0.100000,580.486996,4.189172e+05,1.600000,1.600000,0.504250,10.300000,63.200000
50%,2008.000000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.755000,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000,72.100000
75%,2012.000000,227.000000,22.000000,7.390000,441.534144,96.000000,360.250000,56.100000,28.000000,97.000000,7.330000,97.000000,0.800000,4779.405190,4.584371e+06,7.100000,7.200000,0.772000,14.100000,75.600000
max,2015.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.600000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000,89.000000




I will drop the country column, why? To create a predictive model, knowing from which country data comes can be confusing and it is not a column I can generalize over. I want to learn a general pattern for all the countries, and not only those dependent on specific countries.



In [5]:
dataset = dataset.drop(['Country'], axis=1)

In [6]:
labels = dataset.iloc[:,-1]

In [7]:
features = dataset.iloc[:, 0:-1]

In [8]:
features.head()

,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [9]:
labels.head()

0    65.0
1    59.9
2    59.9
3    59.5
4    59.2
Name: Life expectancy, dtype: float64

## Data Processing

In [10]:
features = pd.get_dummies(features)

In [11]:
features.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_Developed,Status_Developing
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,0,1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,0,1
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,0,1
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,0,1
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,0,1


In [21]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.33, random_state = 42)

In [22]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer 

numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

In [23]:
features_train_scaled = ct.fit_transform(features_train)

In [24]:
features_test_scaled = ct.transform(features_test)

## Building the Model

In [25]:
from tensorflow.keras.models import Sequential

my_model = Sequential()

In [27]:
from tensorflow.keras.layers import InputLayer

#creating the input layer with the shape corresponding to the number of features
input = InputLayer(input_shape=(features.shape[1],))

In [28]:
#adding the input layer to the model
my_model.add(input)

In [29]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [31]:
from tensorflow.keras.layers import Dense

#adding one hidden layer with 64 neurons
my_model.add(Dense(64, activation='relu'))

In [33]:
#adding output layer with a single output
my_model.add(Dense(1))

In [34]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 5,635
Trainable params: 5,635
Non-trainable params: 0
_________________________________________________________________


## Initializing the Optimizer & Compiling the Model

In [35]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.01)

In [37]:
my_model.compile(loss = 'mse', metrics = 'mae', optimizer=opt)

## Fit & Evaluate the Model

In [38]:
my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=1, verbose=1)

Epoch 1/40
1968/1968 [==============================] - 6s 1ms/step - loss: 433.4377 - mae: 12.9265
Epoch 2/40
1968/1968 [==============================] - 3s 1ms/step - loss: 53.9465 - mae: 5.2627
Epoch 3/40
1968/1968 [==============================] - 3s 1ms/step - loss: 29.8414 - mae: 4.1400
Epoch 4/40
1968/1968 [==============================] - 3s 1ms/step - loss: 34.3734 - mae: 4.1201
Epoch 5/40
1968/1968 [==============================] - 3s 1ms/step - loss: 18.9142 - mae: 3.3285
Epoch 6/40
1968/1968 [==============================] - 3s 1ms/step - loss: 23.3455 - mae: 3.5635
Epoch 7/40
1968/1968 [==============================] - 3s 1ms/step - loss: 19.6253 - mae: 3.2652
Epoch 8/40
1968/1968 [==============================] - 3s 1ms/step - loss: 20.7434 - mae: 3.4015
Epoch 9/40
1968/1968 [==============================] - 3s 1ms/step - loss: 16.3131 - mae: 2.9497
Epoch 10/40
1968/1968 [==============================] - 3s 1ms/step - loss: 14.9690 - mae: 2.9398
Epoch 11/40
1968/

In [39]:
res_mse, rse_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

In [40]:
print("MSE ", res_mse)
print("MAE ", rse_mae)

MSE  6.49599027633667
MAE  1.832485556602478


In [42]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(features_train_scaled, labels_train)
y_pred = dummy_regr.predict(features_test_scaled)
MAE_baseline = mean_absolute_error(labels_test, y_pred)
print("MAE baseline: ", MAE_baseline)

MAE baseline:  7.833263766658285


## Results

We can see from the output that the model has performed very well with an mean absoulte error of just over 1.8. When compared to a baseline of 7.8, the model has performed very well. 